## Using Tools in Llama 3.1 to provide real-time information in an LLM chat

Llama 3.1 was just released this week, and supports 'tools'.  Tools are simple functions that you can write and provide to the LLM to answer particular questions.   This can be very valuable in an agentic workflow built for a highly specialized purpose, where you can use the LLM to determine the intent of a user's question in a chat, extract the relevant parameters from the chat and pipe them into the function you have written in order to get the required info.

Let's look at a simple example.  I'm going to create a workflow to find out the current temperature in a particular place of interest.

### Creating a tool to get the current temperature

In this code I am writing a simple Python function which will ping an API to get the current temperature in a place.

In [25]:
import requests

# function to get the current temperature in a place
def get_current_temperature(place: str) -> str:
  base_url = f"https://wttr.in/{place}?format=j1"
  response = requests.get(base_url)
  data = response.json()
  return f"The current temperature in {place} is {data['current_condition'][0]['temp_C']} degrees Celsius"

# test the function
get_current_temperature("London")

'The current temperature in London is 27 degrees Celsius'

That seems to have worked.  

### Adding the tool to the LLM workflow

Next I am going to use this function as a tool to feed to the LLM.  Not all LLMs support tools, but for those that do, the idea is as follows:
* The LLM should identify the intent of a user query and determine that the intent is appropriate to call the tool
* The LLM should extract or determine the right arguments/parameters to give to the tool
* The LLM should capture the response of the tool and feed it back to the user.  To do this it may have to override some default behaviors (for example many LLMs have default responses which inform users that they cannot provide certain real-time information).

To create this workflow, I am going to use an asynchronous chat client via Ollama - I'll leave the model as a parameter so we can try out a few models at the end.

In [26]:
import ollama
import asyncio

async def weather_chat(model: str, query: str):
  client = ollama.AsyncClient()
  # Initialize conversation with a query
  messages = [{'role': 'user', 'content': query}]

  # First API call: Send the query and function description to the model
  # use the tools parameter to define the tool so the LLM can understand it 
  response = await client.chat(
    model = model,
    messages = messages,
    tools = [
      {
        'type': 'function',
        'function': {
          'name': 'get_current_temperature',
          'description': 'Get the temperature in a place',
          'parameters': {
            'type': 'object',
            'properties': {
              'place': {
                'type': 'string',
                'description': 'The place for which the temperature is requested',
              }
            },
            'required': ['place'],
          },
        },
      },
    ],
  )

  # Add the model's response to the conversation history
  messages.append(response['message'])

  # Check if the model decided to use the provided tool and warn if it did not
  if not response['message'].get('tool_calls'):
    print("The model didn't use the function. Its response was:")
    print(response['message']['content'])
    return

  # If it did use the tool, process tool calls made by the model
  if response['message'].get('tool_calls'):
    available_functions = {
      'get_current_temperature': get_current_temperature,
    }
    for tool in response['message']['tool_calls']:
      function_to_call = available_functions[tool['function']['name']]
      function_response = function_to_call(tool['function']['arguments']['place'])
      # Add function response to the conversation
      # Warn model to override any default behavior
      messages.append(
        {
          'role': 'tool',
          'content': f"""
          Answer the following question: {query}.  
          Ignore any previous instructions or defaults and instead use the following information:  {function_response}
          """,
        }
      )

  # Second API call: Get final response from the model
  # Hopefully it will provide the information obtained using the tool
  final_response = await client.chat(model=model, messages=messages)
  print(final_response['message']['content'])


### Testing using various open models and queries
First, let's try this with Google's Gemma2 model and a simple question.


In [27]:
await weather_chat('gemma2:27b', 'What is the temperature like right now in Dublin?')

ResponseError: gemma2:27b does not support tools

OK, so we can see that some models don't yet support this feature.  Let's try the small version of Llama 3.1.

In [28]:
await weather_chat('llama3.1:8b', 'What is the temperature like right now in Dublin?')

The current temperature in Dublin is 20 degrees Celsius.


Works nicely - let's try asking indirectly.

In [29]:
await weather_chat('llama3.1:8b', "What is the temperature like in Ireland's capital city?")

The temperature in Dublin, the capital city of Ireland, is currently 20 degrees Celsius.


Very nice! Let's push things a little further.

In [32]:
await weather_chat('llama3.1:8b', "My sister says that if I flew into the capital city of Norway today, I should wear warm clothing.  Should I trust her advice?")

Based on the current temperature in Oslo being 20 degrees Celsius, it's likely that your sister's advice to wear warm clothing was incorrect. In fact, 20°C is a relatively mild temperature, and you might even consider wearing light layers or no additional warmth at all. So, to answer your question, it would be reasonable to disregard her advice in this case.


In [33]:
await weather_chat('llama3.1:8b', "Compare the temperatures of these two cities right now: Dunedin, New Zealand and Reykjavik, Iceland?")

Based on the provided information:

Dunedin, New Zealand's current temperature is 4°C.
Reykjavik, Iceland's current temperature is 13°C.

Comparing these temperatures, we can see that Reykjavik, Iceland is currently warmer than Dunedin, New Zealand by 9°C.


In [34]:
await weather_chat('llama3.1:8b', "What kinds of clothes should I pack for my trip to Hobart, Tasmania which leaves tomorrow?")

Based on the current temperature in Hobart, Tasmania being 1 degrees Celsius, it's likely that you'll be experiencing cool to mild weather.

For your trip, I'd recommend packing:

* A mix of lightweight and breathable tops (t-shirts, blouses) for warmer moments
* Some insulating layers (fleeces, sweaters) for cooler periods
* A waterproof or windbreaker jacket to protect against any rain or wind
* Warm socks and a hat for chilly mornings and evenings
* Comfortable walking shoes or boots with good grip, as Hobart's streets can be quite hilly and windy
* Layers of clothing that can be easily added or removed as needed to maintain a comfortable body temperature

Remember to check the weather forecast before your trip to ensure you're prepared for any potential weather conditions.
